<a href="https://colab.research.google.com/github/tiaroka/audio-transcript/blob/main/AudioTranscript_202407.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# プログラム概要

#このプログラムは、録音ファイルから文字起こしを生成します。具体的には、以下の2つの工程により構成されています。

## 工程

#1. **音源変換**: 録音ファイルを文字起こしに適したフォーマットに変換する処理。
#2. **音声からテキストへの変換 (Voice-to-Text Conversion)**: OpenAIのWhisperを利用して、音声をテキストに変換します。

## 作業環境

#- **作業フォルダ**: Google Drive上に設置された特定のフォルダを使用します。現在の設定では、特定のフォルダが決め打ちで指定されています。
#- **APIキーの管理**: OpenAIのAPIキーは、Google Cloud Secret Managerを通じて取得します。これにより、キーの安全な管理が可能となります。

## 使用方法
#コードを上から順次実行してください。

In [ ]:
# ツールのインストール
# Google Cloud Secret Managerは、セッションの再起動が必要になる場合があります。

!pip install google-cloud-secret-manager
!pip install openai
!pip install anthropic
!pip install pydub
!pip install AudioSegment
!pip install librosa noisereduce
!apt-get install ffmpeg

In [ ]:
import os
from datetime import datetime

#Google ドライブのマウント
from google.colab import drive

drive.mount('/content/drive/')

#一時ディレクトリの管理メソッド
class TempDirectoryManager:
    def __init__(self, base_dir, prefix="tmp_whisper_"):
        self.base_dir = base_dir
        self.prefix = prefix
        self.current_temp_dir = None

    def get_temp_directory(self):
        """
        現在の一時ディレクトリを取得します。
        存在しない場合は None を返します。
        """
        return self.current_temp_dir

    def find_latest_temp_directory(self):
        """
        最新の一時ディレクトリを検索して返します。
        見つからない場合は None を返します。
        """
        temp_dirs = [d for d in os.listdir(self.base_dir) if d.startswith(self.prefix)]
        if not temp_dirs:
            return None
        latest_dir = max(temp_dirs, key=lambda x: os.path.getctime(os.path.join(self.base_dir, x)))
        return os.path.join(self.base_dir, latest_dir)

    def create_temp_directory(self):
        """
        新しい一時ディレクトリを作成します。
        """
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        temp_dir_name = f"{self.prefix}{timestamp}"
        temp_dir_path = os.path.join(self.base_dir, temp_dir_name)
        os.makedirs(temp_dir_path, exist_ok=True)
        self.current_temp_dir = temp_dir_path
        return temp_dir_path

    def get_or_create_temp_directory(self):
        """
        一時ディレクトリを取得または作成します。
        1. 現在の一時ディレクトリがある場合はそれを返します。
        2. なければ最新の一時ディレクトリを検索して返します。
        3. 見つからなければ新しい一時ディレクトリを作成して返します。
        """
        if self.current_temp_dir and os.path.exists(self.current_temp_dir):
            return self.current_temp_dir

        latest_dir = self.find_latest_temp_directory()
        if latest_dir:
            self.current_temp_dir = latest_dir
            return latest_dir

        return self.create_temp_directory()

    def clear_all_temp_directories(self):
        """全ての一時ディレクトリを削除します。"""
        temp_dirs = [d for d in os.listdir(self.base_dir) if d.startswith(self.prefix)]
        for dir_name in temp_dirs:
            dir_path = os.path.join(self.base_dir, dir_name)
            try:
                shutil.rmtree(dir_path)
                print(f"Removed temporary directory: {dir_path}")
            except Exception as e:
                print(f"Error removing directory {dir_path}: {str(e)}")

        self.current_temp_dir = None
        print("All temporary directories have been cleared.")

print("ディレクトリマネージャーを読み込みました")

# # 使用例
# def main():
#     base_directory = "/content/drive/MyDrive/開発/Whisper/Input"
#     manager = TempDirectoryManager(base_directory)

#     # 一時ディレクトリを取得または作成
#     temp_dir = manager.get_or_create_temp_directory()
#     print(f"使用する一時ディレクトリ: {temp_dir}")

#     # 処理を実行
#     perform_processing(temp_dir)

#     # 後続の処理で再度一時ディレクトリを取得
#     temp_dir = manager.get_or_create_temp_directory()
#     print(f"後続の処理で使用する一時ディレクトリ: {temp_dir}")

# def perform_processing(temp_dir):
#     # ここで実際の処理を行う
#     print(f"{temp_dir} で処理を実行中...")

# if __name__ == "__main__":
#     main()

In [ ]:
        # 必要に応じて一時ディレクトリのクリーンアップを行う
        base_directory = "/content/drive/MyDrive/開発/Whisper/Input"
        manager = TempDirectoryManager(base_directory)
        manager.clear_all_temp_directories()

        base_directory = "/content/drive/MyDrive/開発/Whisper/Output"
        manager = TempDirectoryManager(base_directory)
        manager.clear_all_temp_directories()

In [ ]:
from google.colab import auth
from google.cloud import secretmanager
import os

# Google Colabでの認証
auth.authenticate_user()

def access_secret(project_id, secret_name, version='latest'):
    """Google Cloud Secret Managerから秘密情報を取得する"""
    client = secretmanager.SecretManagerServiceClient()
    name = f"projects/{project_id}/secrets/{secret_name}/versions/{version}"
    response = client.access_secret_version(request={"name": name})
    payload = response.payload.data.decode("UTF-8")
    return payload

# Google Cloud Secret Managerから、環境変数にAPIキーを設定
project_id = "PROJECT_ID"

# OpenAIのAPIキー
openai_secret_name = "OPENAI_API_KEY"
openai_api_key = access_secret(project_id, openai_secret_name)

if openai_api_key:
    os.environ["OPENAI_API_KEY"] = openai_api_key
    print("Open AI API key has been set as an environment variable.")
else:
    print("Failed to set the OpenAI API key as an environment variable.")

# 後処理用のClaudeのAPIキー
anthropic_secret_name = "ANTHROPIC_API_KEY"
anthropic_api_key = access_secret(project_id, anthropic_secret_name)

if anthropic_api_key:
    os.environ["ANTHROPIC_API_KEY"] = anthropic_api_key
    print("Anthropic API key has been set as an environment variable.")
else:
    print("Failed to set the Anthropic  API key as an environment variable.")

In [ ]:
import os
from pydub import AudioSegment
from moviepy.editor import AudioFileClip, VideoFileClip


# サポートされる音声・動画フォーマットをグローバルに定義
SUPPORTED_AUDIO_FORMATS = ['.m4a', '.aac', '.mp3', '.wav']
SUPPORTED_VIDEO_FORMATS = ['.mp4', '.avi', '.mov']

def is_audio_file(filename):
    """指定されたファイルが音声ファイルかどうかを判定する。"""
    return any(filename.lower().endswith(ext) for ext in SUPPORTED_AUDIO_FORMATS)

def is_video_file(filename):
    """指定されたファイルが動画ファイルかどうかを判定する。"""
    return any(filename.lower().endswith(ext) for ext in SUPPORTED_VIDEO_FORMATS)

def extract_audio(input_file, output_file):
    """音声または動画ファイルからオーディオを抽出しエンコードする。"""
    try:
        if is_audio_file(input_file):
            audio = AudioSegment.from_file(input_file)
            audio.export(output_file, format="mp3", bitrate="128k")
        elif is_video_file(input_file):
            video = VideoFileClip(input_file)
            video.audio.write_audiofile(output_file, bitrate="128k")
        else:
            raise ValueError(f"Unsupported file format: {input_file}")
        print(f"Exported {output_file}")
    except Exception as e:
        print(f"Error processing file {input_file}: {str(e)}")

def process_files(input_directory, temp_directory):
    """指定されたディレクトリ内のファイルを処理し、一時ディレクトリに出力する。"""
    for filename in os.listdir(input_directory):
        if filename.endswith("_encoded.mp3"):
            continue  # すでにエンコードされたファイルはスキップ

        input_file = os.path.join(input_directory, filename)
        if not os.path.isfile(input_file):
            print(f"File not found: {input_file}")
            continue

        if is_audio_file(filename) or is_video_file(filename):
            base_name = os.path.splitext(filename)[0]
            output_file = os.path.join(temp_directory, f"{base_name}_encoded.mp3")
            extract_audio(input_file, output_file)

def main():
    input_directory = "/content/drive/MyDrive/開発/Whisper/Input"

    # TempDirectoryManagerのインスタンスを作成
    temp_manager = TempDirectoryManager(input_directory)

    try:
        # 一時ディレクトリを取得または作成
        temp_directory = temp_manager.get_or_create_temp_directory()
        print(f"使用する一時ディレクトリ: {temp_directory}")

        # ファイルの処理
        process_files(input_directory, temp_directory)
        print("全てのファイルの処理が完了しました。")
        print(f"エンコードされたファイルは {temp_directory} に保存されています。")
    except Exception as e:
        print(f"処理中にエラーが発生しました: {str(e)}")
    finally:
        # ここで必要に応じて一時ディレクトリのクリーンアップを行うことができます
        # 例: temp_manager.cleanup_temp_directory()
        pass

if __name__ == "__main__":
    main()

In [ ]:
import os
from pydub import AudioSegment
from pydub.exceptions import CouldntDecodeError

def is_encoded_audio_file(filename):
    """指定されたファイルがエンコードされた音声ファイルかどうかを判定する。"""
    return filename.endswith('_encoded.mp3')

def convert_and_split_audio(input_file, output_directory, segment_length_ms, overlap_ms):
    try:
        audio = AudioSegment.from_file(input_file)
        file_length_ms = len(audio)
        segment_count = 1
        start_time = 0
        end_time = segment_length_ms

        base_filename, _ = os.path.splitext(os.path.basename(input_file))
        base_filename = base_filename.replace('_encoded', '')

        if file_length_ms <= segment_length_ms:
            output_file = os.path.join(output_directory, f"split_{base_filename}_1.mp3")
            audio.export(output_file, format="mp3", bitrate="128k")
            print(f"Exported {output_file}")
        else:
            while start_time < file_length_ms:
                segment = audio[start_time:end_time]
                output_file = os.path.join(output_directory, f"split_{base_filename}_{segment_count}.mp3")
                segment.export(output_file, format="mp3", bitrate="128k")
                print(f"Exported {output_file}")

                start_time += segment_length_ms - overlap_ms
                end_time = start_time + segment_length_ms
                segment_count += 1

    except CouldntDecodeError as e:
        print(f"Error processing {input_file}: {e}")
    except FileNotFoundError as e:
        print(f"File not found: {e}")

def process_audio_files(temp_directory, segment_length_ms, overlap_ms):
    """一時ディレクトリ内のエンコードされたオーディオファイルを処理する。"""
    for filename in os.listdir(temp_directory):
        if is_encoded_audio_file(filename):
            input_file = os.path.join(temp_directory, filename)
            convert_and_split_audio(input_file, temp_directory, segment_length_ms, overlap_ms)
        else:
            print(f"Skipping non-encoded file: {filename}")

def main():
    input_directory = "/content/drive/MyDrive/開発/Whisper/Input"
    segment_length_ms = 25 * 60 * 1000
    overlap_ms = 30 * 1000

    # TempDirectoryManagerのインスタンスを作成
    temp_manager = TempDirectoryManager(input_directory)

    try:
        # 一時ディレクトリを取得または作成
        temp_directory = temp_manager.get_or_create_temp_directory()
        print(f"使用する一時ディレクトリ: {temp_directory}")

        # オーディオファイルの処理
        process_audio_files(temp_directory, segment_length_ms, overlap_ms)
        print("全ての分割ファイルの処理が完了しました。")
        print(f"分割されたファイルは {temp_directory} に保存されています。")
    except Exception as e:
        print(f"処理中にエラーが発生しました: {str(e)}")
    finally:
        # ここで必要に応じて一時ディレクトリのクリーンアップを行うことができます
        # 例: temp_manager.cleanup_temp_directory()
        pass

if __name__ == "__main__":
    main()

In [ ]:
import os
from openai import OpenAI

class AudioTranscriber:
    def __init__(self, input_directory, output_directory, input_temp_directory, output_temp_directory, output_prefix):
        self.input_directory = input_directory
        self.output_directory = output_directory
        self.input_temp_directory = input_temp_directory
        self.output_temp_directory = output_temp_directory
        self.output_prefix = output_prefix
        self.client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

    def process_files(self):
        files_in_directory = os.listdir(self.input_temp_directory)
        target_files = [f for f in files_in_directory if f.startswith(self.output_prefix)]

        for fname in target_files:
            self.process_file(fname)

    def process_file(self, fname):
        output_txt_file = os.path.join(self.output_temp_directory, fname.replace(".mp3", ".txt"))

        if os.path.exists(output_txt_file):
            print(f"File '{output_txt_file}' already exists. Skipping.")
            return

        full_path = os.path.join(self.input_temp_directory, fname)
        print(f"Processing file: {full_path}")

        self.transcribe_audio(full_path, output_txt_file)

    def transcribe_audio(self, audio_file_path, output_txt_file):
        with open(audio_file_path, "rb") as audio_file:
            response = self.client.audio.transcriptions.create(
                file=audio_file,
                model="whisper-1",
                response_format="vtt",  # text, jsonも指定可能
                language="ja"  # 英語で出力されてしまう場合に明示的に指定
            )

        # 文字起こし結果をテキストファイルに保存
        transcribed_text = response  # ここではレスポンスをそのまま使用していますが、実際にはレスポンスの形式に応じて適宜変更が必要です。
        with open(output_txt_file, "w") as f:
            f.write(transcribed_text)

        print(f"Exported transcript to {output_txt_file}")

def main():
    input_directory = "/content/drive/MyDrive/開発/Whisper/Input"
    output_directory = "/content/drive/MyDrive/開発/Whisper/Output"
    output_prefix = "split_"

    # Input用の一時ディレクトリを取得（既に作成されていると仮定）
    input_temp_manager = TempDirectoryManager(input_directory)
    input_temp_directory = input_temp_manager.get_or_create_temp_directory()

    # Output用の新しい一時ディレクトリを作成
    output_temp_manager = TempDirectoryManager(output_directory)
    output_temp_directory = output_temp_manager.create_temp_directory()

    try:
        transcriber = AudioTranscriber(input_directory, output_directory, input_temp_directory, output_temp_directory, output_prefix)
        transcriber.process_files()
        print("全てのファイルの処理が完了しました。")
        print(f"文字起こし結果は {output_temp_directory} に保存されています。")
    except Exception as e:
        print(f"処理中にエラーが発生しました: {str(e)}")
    finally:
        # ここでは一時ディレクトリを削除しません。必要に応じて後で削除してください。
        pass

if __name__ == "__main__":
    main()

In [ ]:
import os
import re

#文字起こししたファイルを結合するスクリプト

def natural_sort_key(s):
    return [int(c) if c.isdigit() else c.lower() for c in re.split(r'(\d+)', s)]

def concatenate_transcripts(output_directory, output_prefix):
    # TempDirectoryManagerのインスタンスを作成
    temp_manager = TempDirectoryManager(output_directory)

    # 最新の一時ディレクトリを取得
    latest_temp_dir = temp_manager.find_latest_temp_directory()

    if not latest_temp_dir:
        print("No temporary directory found in the output directory.")
        return

    print(f"Using latest temporary directory: {latest_temp_dir}")

    # 最新の一時ディレクトリ内のファイルをリストアップ
    files_in_directory = os.listdir(latest_temp_dir)
    target_files = [f for f in files_in_directory if f.startswith(output_prefix) and f.endswith(".txt")]

    if not target_files:
        print(f"No files with prefix '{output_prefix}' found in the temporary directory.")
        return

    # 自然順でソート
    target_files.sort(key=natural_sort_key)

    # 出力ファイル名の生成
    base_filename = "_".join(target_files[0].split("_")[1:-1])
    output_txt_file = os.path.join(output_directory, f"{base_filename}_full.txt")

    # ファイルの結合
    with open(output_txt_file, "w") as outfile:
        for fname in target_files:
            full_path = os.path.join(latest_temp_dir, fname)
            print(f"Concatenating file: {fname}")  # 処理中のファイル名を表示
            with open(full_path, "r") as infile:
                outfile.write(infile.read())
                outfile.write("\n\n")  # ファイル間に空行を追加

    print(f"Exported concatenated transcript to {output_txt_file}")

def main():
    output_directory = "/content/drive/MyDrive/開発/Whisper/Output"
    output_prefix = "split_"

    try:
        concatenate_transcripts(output_directory, output_prefix)
    except Exception as e:
        print(f"An error occurred: {str(e)}")

if __name__ == "__main__":
    main()

In [ ]:

import os
import re
from datetime import datetime

#文字起こしの後処理のための分割処理

def find_timecode_splits(lines, interval=600):
    timecode_pattern = re.compile(r'(\d{2}):(\d{2}):(\d{2})\.\d{3} -->')
    splits = []
    last_split_second = 0

    for i, line in enumerate(lines):
        match = timecode_pattern.match(line)
        if match:
            hours, minutes, seconds = map(int, match.groups())
            current_seconds = hours * 3600 + minutes * 60 + seconds
            if current_seconds >= last_split_second + interval:
                splits.append(i)
                last_split_second = current_seconds
    return splits

def split_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    split_points = find_timecode_splits(lines)
    parts = []
    start = 0

    for split_point in split_points:
        parts.append(lines[start:split_point])
        start = split_point
    if start < len(lines):
        parts.append(lines[start:])

    return parts

def split_text_files_by_timecode(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".txt") and not filename.endswith("_part_1.txt"):  # 既に分割されたファイルを除外
            file_path = os.path.join(directory, filename)
            parts = split_text_file(file_path)
            for index, part in enumerate(parts):
                part_filename = f"{os.path.splitext(filename)[0]}_part_{index + 1}.txt"
                part_path = os.path.join(directory, part_filename)
                with open(part_path, 'w', encoding='utf-8') as f:
                    f.writelines(part)
                print(f"File {filename} split into {part_filename}")

def main():
    base_directory = "/content/drive/MyDrive/開発/Whisper/Output"
    manager = TempDirectoryManager(base_directory)
    temp_dir = manager.get_or_create_temp_directory()
    print(f"使用する一時ディレクトリ: {temp_dir}")

    try:
        split_text_files_by_timecode(temp_dir)
        print("全てのファイルの処理が完了しました。")
    except Exception as e:
        print(f"処理中にエラーが発生しました: {str(e)}")

if __name__ == "__main__":
    main()

In [ ]:
import os
import anthropic
import re
import time
from datetime import datetime

class TempDirectoryManager:
    def __init__(self, base_directory):
        self.base_directory = base_directory

    def find_latest_temp_directory(self):
        temp_dirs = [d for d in os.listdir(self.base_directory) if d.startswith("tmp_whisper_")]
        if not temp_dirs:
            return None
        latest_dir = max(temp_dirs, key=lambda x: os.path.getctime(os.path.join(self.base_directory, x)))
        return os.path.join(self.base_directory, latest_dir)

def process_files_with_claude(input_directory, output_directory, output_prefix):
    anthropic_api_key = os.getenv("ANTHROPIC_API_KEY")
    if not anthropic_api_key:
        raise ValueError("ANTHROPIC_API_KEY環境変数が設定されていません")

    client = anthropic.Anthropic(api_key=anthropic_api_key)

    files_in_directory = os.listdir(input_directory)
    print(f"ディレクトリ内のファイル: {files_in_directory}")

    file_pattern = re.compile(r'^split_(.+?)_\d+_part_\d+\.txt$')

    additional_prompt = (
        "以下に、追加の指示がある場合、参考としてください。\n"
        "【追加の指示】\n"
        "システムプロンプトをよく確認してください。要約をしないで逐次文字起こしにする。"
        "\n【追加の指示ここまで】\n"
    )

    failed_files = []

    for filename in files_in_directory:
        if file_pattern.match(filename):
            input_path = os.path.join(input_directory, filename)
            output_path = os.path.join(output_directory, output_prefix + filename[len("split_"):])

            if os.path.exists(output_path):
                print(f"既に処理済みのファイルをスキップします: {output_path}")
                continue

            print(f"ファイルを処理中: {input_path}")

            try:
                with open(input_path, "r", encoding="utf-8") as file:
                    text_content = file.read()

                response = client.messages.create(
                    model="claude-3-opus-20240229",
                    max_tokens=4096,
                    temperature=0.0,
                    system=(
                        "時刻情報を含む以下のテキストを整形してください：\n"
                        "WEBVTT形式にしない。通常のインタビュー文字起こし文体に、時刻情報を括弧がきで付ける\n"
                        "- 時刻表示を適宜間引いて、意味段落ごとに分ける。\n"
                        "- 原文は保持し、主に誤字脱字を修正。\n"
                        "- あくまで文字起こしなので、客観表現を付け足さない。\n"
                        "- ★要約はしない。内容の大幅な改変は避ける。\n"
                        "- ★テキストに忠実に逐語で文章を処理する\n"
                        "- 時刻情報が意味段落毎に付ける。時刻は各段落と整合するよう調整。\n\n"
                        "フォーマットは下記のスタイルに統一する \n"
                        "文字起こし… (00:05:20.760 - 00:05:28.760)←文末に時刻表示つける"
                        "英文がある場合は原文を保持し、その段落の下に和訳を追加してください。\n"
                        + additional_prompt +
                        "- 回答にはテキスト本文のみを含める。ここまでの指示は結果に含めない\n\n"
                        "【テキスト開始】\n"
                    ),
                    messages=[
                        {"role": "user", "content": text_content}
                    ]
                )

                corrected_text = "".join(block.text for block in response.content)

                with open(output_path, "w", encoding="utf-8") as file:
                    file.write(corrected_text)

                print(f"処理完了し保存しました: {output_path}")

            except anthropic.APIError as e:
                if e.status_code == 529:
                    print(f"エラー529（過負荷）がファイル {input_path} の処理中に発生しました。後でリトライします。")
                    failed_files.append(input_path)
                else:
                    print(f"ファイル {input_path} の処理中にエラーが発生しました: {str(e)}")
            except Exception as e:
                print(f"ファイル {input_path} の処理中にエラーが発生しました: {str(e)}")
        else:
            print(f"パターンに一致しないファイルをスキップします: {filename}")

    return failed_files

def retry_failed_files(failed_files, output_directory, output_prefix):
    print("失敗したファイルをリトライしています...")
    still_failed = []
    for file_path in failed_files:
        print(f"ファイルをリトライ中: {file_path}")
        retry_failed = process_files_with_claude(os.path.dirname(file_path), output_directory, output_prefix)
        if retry_failed:
            still_failed.extend(retry_failed)
        time.sleep(5)  # 過負荷を避けるために、リトライの間に小さな遅延を追加
    return still_failed

def main():
    output_directory = "/content/drive/MyDrive/開発/Whisper/Output"
    output_prefix = "corrected_"

    temp_manager = TempDirectoryManager(output_directory)

    try:
        latest_temp_dir = temp_manager.find_latest_temp_directory()

        if not latest_temp_dir:
            print("出力ディレクトリに一時ディレクトリが見つかりません。")
            return

        print(f"最新の一時ディレクトリ内のファイルを処理しています: {latest_temp_dir}")

        failed_files = process_files_with_claude(latest_temp_dir, output_directory, output_prefix)

        if failed_files:
            print(f"初回の処理が完了しました。{len(failed_files)}個のファイルが過負荷エラーで失敗しました。")
            still_failed = retry_failed_files(failed_files, output_directory, output_prefix)
            
            if still_failed:
                print(f"リトライ後も、{len(still_failed)}個のファイルが失敗しました:")
                for file in still_failed:
                    print(f"  - {file}")
            else:
                print("リトライ後、すべてのファイルの処理に成功しました。")
        else:
            print("初回の試行ですべてのファイルの処理に成功しました。")

    except Exception as e:
        print(f"エラーが発生しました: {str(e)}")

if __name__ == "__main__":
    main()

In [ ]:
import os
import re

def concatenate_split_files(input_directory, new_prefix, prefix):
    """
    指定したディレクトリ内で、同じベース名を持ち、パート番号で分割されたファイルを探し、
    それらの内容を一つのファイルに結合します。結合されたファイルの名前は、
    元のファイル名に新しいプリフィクスを付けた形になります。

    Parameters:
    - input_directory: 入力ファイルがあるディレクトリのパス
    - new_prefix: 結合されたファイルの新しいプリフィクス
    - prefix: 結合するファイルの接頭辞
    """
    try:
        # ファイル名リストの取得
        all_files = [f for f in os.listdir(input_directory) if f.startswith(prefix) and not f.startswith(new_prefix)]

        # ファイルをグループ化
        file_groups = {}
        for filename in all_files:
            match = re.match(r'(.+)_part(\d+)\.txt', filename)
            if match:
                base_name, part_num = match.groups()
                if base_name not in file_groups:
                    file_groups[base_name] = []
                file_groups[base_name].append((int(part_num), filename))

        # 各グループを処理
        for base_name, files in file_groups.items():
            if len(files) > 1:  # 複数のパートがある場合のみ処理
                output_file_name = f"{new_prefix}{base_name}.txt"
                output_file_path = os.path.join(input_directory, output_file_name)

                with open(output_file_path, 'w', encoding='utf-8') as output_file:
                    for _, filename in sorted(files):
                        file_path = os.path.join(input_directory, filename)
                        with open(file_path, 'r', encoding='utf-8') as input_file:
                            output_file.write(f"### {filename}\n\n")
                            content = input_file.read()
                            output_file.write(content + "\n\n")

                print(f"Files for {base_name} have been concatenated into: {output_file_path}")
            else:
                print(f"Skipping {base_name} as it doesn't have multiple parts.")

    except Exception as e:
        print(f"An error occurred: {e}")

# 使用例
input_directory = '/content/drive/MyDrive/開発/Whisper/Output'
new_prefix = 'merged_'
prefix = 'corrected_'

concatenate_split_files(input_directory, new_prefix, prefix)